Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras import models, layers 
from sklearn.model_selection import train_test_split   
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
image_info = pd.read_csv('../processed_data/ISIC_2019_Training_GroundTruth_Processed_Balanced.csv')

In [3]:
image_info

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,Cat,y
0,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
1,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL,1
2,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
3,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL,1
4,ISIC_0000007,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
...,...,...,...,...,...,...,...,...,...,...,...,...
24801,ISIC_0073153_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24802,ISIC_0073157_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24803,ISIC_0073198_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24804,ISIC_0073214_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1


In [4]:
def get_image(df):
    return np.asarray(Image.open((f"../processed_data/{df['Cat']}/{df['image']}.jpg")), dtype=np.float32)

In [5]:
data_size =1000

In [6]:
X = image_info[:data_size].apply(get_image, axis=1)
X = np.stack(X)

In [7]:
y = image_info['y'][:data_size]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Transfered model -VGG

In [9]:
def load_model():
    model = VGG16(weights="imagenet", include_top=False, input_shape=X_train[0].shape)
    return model

In [10]:
def set_nontrainable_layers(model):
    # Set the first layers to be untrainable
    model.trainable = False
    
    return model

In [11]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(100, activation='relu')
    prediction_layer = layers.Dense(1, activation='sigmoid')
    
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
  
    return model

In [12]:
def build_model():
       
    model = load_model()
    model = add_last_layers(model)
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy', 'Recall'])
    return model

In [13]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 12, 12, 512)       14714688  
                                                                 
 flatten (Flatten)           (None, 73728)             0         
                                                                 
 dense (Dense)               (None, 100)               7372900   
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 22,087,689
Trainable params: 7,373,001
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
X_train = preprocess_input(X_train) 
X_test = preprocess_input(X_test)

In [15]:
model = build_model()

es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 5, 
                   verbose = 1, 
                   restore_best_weights = True)


In [16]:
history = model.fit(X_train, y_train, 
                    validation_split=0.3, 
                    epochs=50, 
                    batch_size=8, 
                    callbacks=[es])

Epoch 1/10
62/62 [==============================] - ETA: 0s - loss: 1.2341 - accuracy: 0.6503 - recall: 0.4395

: 

: 

### Evaluate our CNN ###

print(model_dummy.evaluate(X_test, y_test, verbose=1))